# Processing NYC yellow taxi trip data

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import re
from shapely.geometry import Point, LineString

### 1. Extracting trips between 2AM and 3AM on Saturdays and Sundays, and finding number of trips between each O-D pair for each month

In [ ]:
#initialize empty list to store each month's trips, which will be concatenated later
months = []

def get_last_call_trips(month_num:int) -> list:
    '''Extracts trips between 2AM and 3AM on Saturdays and Sundays'''
    
    if month_num < 10:
        fname = 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-0' + str(month_num) + '.csv'
    else:
        fname = 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-' + str(month_num) + '.csv'
        
    print(f"Loading dataset {month_num}/12")
    #load full taxi trip dataset for a month
    full = pd.read_csv(fname, usecols=['tpep_pickup_datetime', 'PULocationID', 'DOLocationID'],
                      parse_dates=['tpep_pickup_datetime'],
                      infer_datetime_format=True)
    
    #set index to pickup datetime
    full.set_index('tpep_pickup_datetime', inplace=True)
    
    #isolate trips that are between 2am and 3am
    month = full.between_time('02:00', '03:00').copy()
    del(full) #delete full dataset to save memory
    
    #remove trips within the same zone
    month.drop(month[month['PULocationID'] == month['DOLocationID']].index, inplace=True)
    
    #isolate weekend trips
    month['day'] = month.index.dayofweek
    month.drop(month[(month['day'] != 5) & (month['day'] != 6)].index, inplace=True)
    
    #find number of trips for each flow
    trips = month.groupby(['PULocationID', 'DOLocationID'], as_index=False).size()
    
    trips['month'] = month_num
    
    #add to months list
    months.append(trips)
    
    #delete month dataset
    del(month)

#Run trip extraction function for each month
for i in range(1,13):
    get_last_call_trips(i)

#Concatenate list of monthly dfs into one df
df = pd.concat(months, ignore_index=True)

#drop rides that begin or end in zones 264 and 265, which are unknown - 521 trips total
df.drop(df[
    (df['PULocationID'] == 264) |
    (df['PULocationID'] == 265) |
    (df['DOLocationID'] == 264) |
    (df['DOLocationID'] == 265)
].index, inplace=True)

### 2. Creating a georeferenced flow dataset

In [ ]:
taxi_zones = gpd.read_file('data/taxi_zones/taxi_zones.shp') #downloaded from https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip
taxi_zones.set_index('LocationID', inplace=True)

#Find centroids of taxi zones (+ coordinates)
centroids = taxi_zones['geometry'].centroid

#Add coordinates of pickup zone centroid to trip df
df = pd.merge(df, centroids.to_frame(), how='left', left_on='PULocationID', right_index=True).rename(columns={0: 'PU_coords'}, inplace=True)

#Add coordinates of dropoff zone centroid to trip df
df = pd.merge(df, centroids.to_frame(), how='left', left_on='DOLocationID', right_index=True).rename(columns={0: 'DO_coords'}, inplace=True)

#Drop 10 records that don't have DO location IDs
df.drop(df[df['DO_coords'].isna()].index, inplace=True) 

#Add new field encoding PU_coords and DO_coords as linestring
df['geometry'] = df.apply(lambda x: LineString([x['PU_coords'], x['DO_coords']]), axis=1)

#Drop pickup and dropoff centroid columns, as these are no longer necessary
df.drop(columns=['PU_coords', 'DO_coords'], inplace=True)

#Convert df to geo-df
gdf = gpd.GeoDataFrame(df,
                geometry='geometry',
                crs='epsg:2263')

#Reproject to Web Mercator
gdf_web_merc = gdf.to_crs('epsg:3857')

### 3. Finding area-weighted average income of drop-off locations for each flow

In [ ]:
#Load income data for New York state Census tracts
income_tract = pd.read_csv('data/ACSST5Y2019.S1903_2021-03-02T112951/ACSST5Y2019.S1903_data_with_overlays_2021-03-02T112740.csv',
                          skiprows=[1], usecols=['GEO_ID', 'NAME', 'S1903_C03_001E'])

#Select only Census tracts in NYC
income_nyc = income_tract.loc[
    income_tract['NAME'].str.contains(r'New York County|Kings County|Bronx County|Richmond County|Queens County')
].copy()
del(income_tract) #save memory by deleting full NY dataset

#Drop non-residential Census tracts
income_nyc.drop(index=income_nyc[income_nyc['S1903_C03_001E'] == '-'].index, inplace=True)

#Convert income column to int
income_nyc['S1903_C03_001E'] = income_nyc['S1903_C03_001E'].str.replace('+','').str.replace(',','').astype('int64')

#Load shapefile for census tracts
tracts_shp = gpd.read_file('https://opendata.arcgis.com/datasets/7bba09631bd740f49ba0442f9603fa38_0.geojson')

#Income dataset and tract shapefile have different codes for each tract, but the codes used in the shapefile can be extracted from
#the GEO_ID column in the income dataset
def get_boroCT(row):
    if re.search('Richmond County|New York County', row['NAME']):
        return row['GEO_ID'][-7:]
    else:
        CT_code = row['GEO_ID'][-6:]
        if 'Bronx County' in row['NAME']:
            return '2' + CT_code
        elif 'Queens County' in row['NAME']:
            return '4' + CT_code
        elif 'Kings County' in row['NAME']:
            return '3' + CT_code
        
income_nyc['BoroCT2010'] = income_nyc.apply(get_boroCT, axis=1)

#Join income data to shapefile
tracts_income_geo = pd.merge(tracts_shp, income_nyc, on='BoroCT2010', how='left')

#Calculate area of each taxi zone
taxi_zones['zone_area'] = taxi_zones.area

#Reproject Census tracts to projection of taxi zones
tracts_income_geo = tracts_income_geo.to_crs('epsg:2263')

#Find intersecting polygons between Census tracts and taxi zones
tract_zone_inter = gpd.overlay(tracts_income_geo, taxi_zones, how='intersection')

#Drop polygons w same geometry
tract_zone_inter.drop_duplicates(subset=['geometry'] ,inplace=True)

#Find area of each polygon
tract_zone_inter['polygon_area'] = tract_zone_inter['geometry'].area

#Divide each polygon's area by the area of the taxi zone, then multiply this proportion by the tract's median income
tract_zone_inter['pc_zone'] = tract_zone_inter.apply(lambda x: (x['polygon_area']/x['zone_area'])*x['S1903_C03_001E'], axis=1)

#Sum income fractions for each taxi zone
zone_income = tract_zone_inter.groupby('LocationID').agg({'pc_zone': 'sum'})

#Using zone_income, add income of drop-off taxi zones to flow dataset
agg_flows_w_income = pd.merge(gdf_web_merc, zone_income, left_on='DOLocationID', right_index=True, how='left').rename(columns={'pc_zone': 'DO_income'})

### 4. Saving flow dataset as shapefile for use with Mapbox

In [ ]:
agg_flows_w_income.to_file('agg_flows_w_income.shp', driver='ESRI Shapefile')